## 모든 레시피의 ID값을 리스트로 반환

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

def get_all_recipe_links():
    """
    '만개의 레시피' 사이트에서 전체 항목의 모든 페이지를 크롤링하여 'common_sp_link' 클래스의 href 속성 중 레시피 ID만 반환.

    Returns:
        list: 모든 페이지의 레시피 ID 값 리스트
    """

    base_url = "https://www.10000recipe.com/recipe/list.html"
    recipe_ids = []

    try:
        for page in range(1, 10000):
            if page % 100 == 0:
                print(f"Crawling page {page}...")

            page_url = f"{base_url}?order=reco&page={page}"
            for _ in range(3):  # 최대 3번 재시도
                try:
                    response = requests.get(page_url)
                    response.raise_for_status()
                    break  # 요청 성공 시 루프 종료
                except requests.RequestException as e:
                    print(f"Retrying page {page} due to error: {e}")
                    time.sleep(2)  # 재시도 간 딜레이
            else:
                print(f"Skipping page {page} after multiple failures.")
                continue

            soup = BeautifulSoup(response.text, 'html.parser')
            links = soup.find_all('a', class_='common_sp_link')
            ids = [link['href'].split('/')[-1] for link in links if 'href' in link.attrs]

            if not ids:
                print(f"No links found on page {page}. Stopping search.")
                break

            recipe_ids.extend(ids)
            time.sleep(0.2)  # 요청 간 딜레이

        return recipe_ids
    except Exception as e:
        print(f"Error during scraping: {e}")
        return []

result_ids = get_all_recipe_links()
print(len(result_ids))
print(result_ids)

Crawling page 100...
Retrying page 133 due to error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Crawling page 200...
Crawling page 300...
Crawling page 400...
Crawling page 500...
Crawling page 600...
Crawling page 700...
Retrying page 751 due to error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Crawling page 800...
Crawling page 900...
Crawling page 1000...
Crawling page 1100...
Crawling page 1200...
Crawling page 1300...
Crawling page 1400...
Crawling page 1500...
Crawling page 1600...
Crawling page 1700...
Crawling page 1800...
Crawling page 1900...
Crawling page 2000...
Crawling page 2100...
Crawling page 2200...
Crawling page 2300...
Crawling page 2400...
Crawling page 2500...
Crawling page 2600...
Crawling page 2700...
Crawling page 2800...
Crawling page 2900...
Crawling page 3000...
Crawling page 3100...
Retrying page 3127 due to error: HTTPSConnectionPool(host='www.10000recipe.co

## 얻은 레시피 ID 값들을 파일로 저장

In [ ]:
import csv

# CSV 파일로 저장
with open('recipe_id.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for recipe_id in result_ids:
        writer.writerow([recipe_id])

print("Result IDs have been saved to result_ids.csv")

Result IDs have been saved to result_ids.csv


## 레시피 정보, 레시피 재료, 레시피 조리도구 테이블 생성

**레시피 정보 데이터프레임**

recipe_id, recipe_name, summary, amount, cooking_time, difficulty_level

<br>

**레시피 재료 데이터프레임**

recipe_id, ingredient_name, ingredient_amount

<br>

**레시피 조리도구 데이터프레임**

recipe_id, utensil

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def fetch_recipe_page(recipe_id, max_retries=3):
    """
    주어진 레시피 ID에 해당하는 페이지를 요청하고 BeautifulSoup 객체를 반환.
    요청 간 0.2초 텀을 주며, 오류 발생 시 최대 3번까지 재시도.

    Parameters:
        recipe_id (str): 레시피 ID
        max_retries (int): 최대 재시도 횟수 (기본값 3)

    Returns:
        BeautifulSoup: 파싱된 HTML 페이지
    """
    base_url = "https://www.10000recipe.com/recipe/"
    url = f"{base_url}{recipe_id}"

    for attempt in range(max_retries):
        try:
            response = requests.get(url)
            response.raise_for_status()
            time.sleep(0.2)  # 요청 간 0.2초 텀 추가
            return BeautifulSoup(response.text, 'html.parser')
        except requests.RequestException as e:
            print(f"Error fetching recipe {recipe_id} (Attempt {attempt + 1}): {e}")
            time.sleep(0.2)  # 재시도 전에도 0.2초 텀 추가
    print(f"Failed to fetch recipe {recipe_id} after {max_retries} attempts.")
    return None

def parse_recipe_info(soup, recipe_id):
    """
    레시피 정보를 추출하여 딕셔너리로 반환.

    Parameters:
        soup (BeautifulSoup): 파싱된 HTML 페이지
        recipe_id (str): 레시피 ID

    Returns:
        dict: 레시피 정보
    """
    if soup is None:
        return None

    recipe_name = soup.find('h3').get_text(strip=True) if soup.find('h3') else "N/A"
    summary = soup.find('div', id='recipeIntro').get_text(strip=True) if soup.find('div', id='recipeIntro') else "N/A"
    amount = soup.find('span', class_='view2_summary_info1').get_text(strip=True) if soup.find('span', class_='view2_summary_info1') else "N/A"
    cooking_time = soup.find('span', class_='view2_summary_info2').get_text(strip=True) if soup.find('span', class_='view2_summary_info2') else "N/A"
    difficulty_level = soup.find('span', class_='view2_summary_info3').get_text(strip=True) if soup.find('span', class_='view2_summary_info3') else "N/A"

    return {
        "recipe_id": recipe_id,
        "recipe_name": recipe_name,
        "summary": summary,
        "amount": amount,
        "cooking_time": cooking_time,
        "difficulty_level": difficulty_level
    }

def parse_ingredients(soup, recipe_id):
    """
    재료 정보를 추출하여 리스트로 반환.

    Parameters:
        soup (BeautifulSoup): 파싱된 HTML 페이지
        recipe_id (str): 레시피 ID

    Returns:
        list: 재료 정보 딕셔너리의 리스트
    """
    if soup is None:
        return []

    ingredient_data = []
    ingredient_section = soup.find('div', id='divConfirmedMaterialArea')
    if ingredient_section:
        ingredients = ingredient_section.find_all('div', class_='ingre_list_name')
        ingredient_amounts = ingredient_section.find_all('span', class_='ingre_list_ea')

        for ingre, amount in zip(ingredients, ingredient_amounts):
            ingredient_data.append({
                "recipe_id": recipe_id,
                "ingredient_name": ingre.get_text(strip=True),
                "ingredient_amount": amount.get_text(strip=True)
            })
    return ingredient_data

def parse_utensils(soup, recipe_id):
    """
    조리도구 정보를 추출하여 리스트로 반환.

    Parameters:
        soup (BeautifulSoup): 파싱된 HTML 페이지
        recipe_id (str): 레시피 ID

    Returns:
        list: 조리도구 정보 딕셔너리의 리스트
    """
    if soup is None:
        return []

    utensil_data = []
    all_items = soup.find_all('div', class_='ingre_list_name')
    for item in all_items:
        # 상위 노드가 id="divConfirmedMaterialArea"가 아닌 경우만 조리도구로 간주
        if not item.find_parent('div', id='divConfirmedMaterialArea'):
            utensil_data.append({
                "recipe_id": recipe_id,
                "utensil": item.get_text(strip=True)
            })
    return utensil_data

def scrape_recipe_details_and_ingredients(recipe_ids):
    """
    주어진 레시피 ID 리스트를 사용하여 각 레시피의 상세 정보, 재료 정보, 조리도구 정보를 데이터프레임으로 반환.
    100번째 반복마다 데이터를 CSV 파일로 저장.

    Parameters:
        recipe_ids (list): 레시피 ID 리스트

    Returns:
        tuple: (레시피 정보 데이터프레임, 재료 정보 데이터프레임, 조리도구 정보 데이터프레임)
    """
    recipe_data = []
    ingredient_data = []
    utensil_data = []

    for id, recipe_id in enumerate(recipe_ids):
        if id % 100 == 0 and id > 0:
            print(f"Processing recipe {id}/{len(recipe_ids)}...")

            # 100번째 반복마다 데이터를 임시로 CSV 파일로 저장 (utf-8-sig 인코딩)
            recipe_df = pd.DataFrame(recipe_data)
            ingredient_df = pd.DataFrame(ingredient_data)
            utensil_df = pd.DataFrame(utensil_data)

            recipe_df.to_csv(f"recipe_data_partial.csv", index=False, encoding='utf-8-sig')
            ingredient_df.to_csv(f"ingredient_data_partial.csv", index=False, encoding='utf-8-sig')
            utensil_df.to_csv(f"utensil_data_partial.csv", index=False, encoding='utf-8-sig')
            print(f"Saved intermediate data at iteration {id}.")

        soup = fetch_recipe_page(recipe_id)

        # 레시피 정보 추가
        recipe_info = parse_recipe_info(soup, recipe_id)
        if recipe_info:
            recipe_data.append(recipe_info)

        # 재료 정보 추가
        ingredient_data.extend(parse_ingredients(soup, recipe_id))

        # 조리도구 정보 추가
        utensil_data.extend(parse_utensils(soup, recipe_id))

    # 최종 데이터프레임 생성
    recipe_df = pd.DataFrame(recipe_data)
    ingredient_df = pd.DataFrame(ingredient_data)
    utensil_df = pd.DataFrame(utensil_data)

    # 최종 데이터 저장 (utf-8-sig 인코딩)
    recipe_df.to_csv("recipe_data_final.csv", index=False, encoding='utf-8-sig')
    ingredient_df.to_csv("ingredient_data_final.csv", index=False, encoding='utf-8-sig')
    utensil_df.to_csv("utensil_data_final.csv", index=False, encoding='utf-8-sig')
    print("Saved final data to CSV.")

    return recipe_df, ingredient_df, utensil_df



# 함수 사용 예제
recipe_ids = result_ids[:10000]
recipe_df, ingredient_df, utensil_df = scrape_recipe_details_and_ingredients(recipe_ids)

# 출력 결과 확인
print("Recipe DataFrame:")
print(recipe_df)
print("\nIngredient DataFrame:")
print(ingredient_df)
print("\nUtensil DataFrame:")
print(utensil_df)

Processing recipe 100/10000...
Saved intermediate data at iteration 100.
Processing recipe 200/10000...
Saved intermediate data at iteration 200.
Processing recipe 300/10000...
Saved intermediate data at iteration 300.
Processing recipe 400/10000...
Saved intermediate data at iteration 400.
Processing recipe 500/10000...
Saved intermediate data at iteration 500.
Processing recipe 600/10000...
Saved intermediate data at iteration 600.
Error fetching recipe 6949975 (Attempt 1): ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Processing recipe 700/10000...
Saved intermediate data at iteration 700.
Processing recipe 800/10000...
Saved intermediate data at iteration 800.
Processing recipe 900/10000...
Saved intermediate data at iteration 900.
Processing recipe 1000/10000...
Saved intermediate data at iteration 1000.
Processing recipe 1100/10000...
Saved intermediate data at iteration 1100.
Processing recipe 1200/10000...
Saved intermediate data a

시간이 너무 오래 걸려 10000개의 레시피만 스크래핑하였습니다.